#良性/恶性乳腺癌肿瘤预测

##1 数据预处理

In [3]:
import pandas as pd
import numpy as np
import sklearn

In [12]:
#创建特征列表
column_names = ['Sample code number', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape',
        'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
        'Normal Nucleoli', 'Mitoses', 'Class']
data=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data',names=column_names)
#将？替换成标准缺失值
data=data.replace(to_replace='?',value=np.nan)
#丢弃带有缺失值的数据(只要一个维度有缺失)
data=data.dropna(how='any')
data.shape

(683, 11)

In [13]:
data

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [17]:
x=data.iloc[:,1:-1]#特征
y=data['Class']#标签
print(x.shape)
print(y.shape)

(683, 9)
(683,)


In [24]:
#对标记数据进行分割，25%作为测试集，75%作为训练集
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)
#使用线性分类模型进行预测
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,SGDClassifier
#标准化数据,报增每个维度的特征数据方差为1，均值为0，使得预测结果不会被某些维度过大的特征值而主导
ss=StandardScaler()
x_train=ss.fit_transform(x_train)#对训练集进行标准化处理
x_test=ss.transform(x_test)   #用训练集求得的均值与方差标准化测试集

In [26]:
#初始化logistic回归和随机梯度下降分类模型
lr=LogisticRegression()
sgdc=SGDClassifier()
#训练模型
lr.fit(x_train,y_train)
#预测
lr_y_predict=lr.predict(x_test)
sgdc.fit(x_train,y_train)
#使用训练好的模型sgdc对x_test预测
sgdc_y_predict=sgdc.predict(x_test)

In [27]:
#性能分析
from sklearn.metrics import classification_report
print(f"Acc of LR Classifier",lr.score(x_test,y_test))
print(classification_report(y_test,lr_y_predict,target_names=['Benign','Malignant']))

Acc of LR Classifier 0.9473684210526315
              precision    recall  f1-score   support

      Benign       0.96      0.96      0.96       113
   Malignant       0.92      0.93      0.92        58

    accuracy                           0.95       171
   macro avg       0.94      0.94      0.94       171
weighted avg       0.95      0.95      0.95       171



In [28]:
#使用随机梯度下降模型自带的评分函数score获得模型在测试集上的准确性结果
print(f"Acc of SGD Classifier",sgdc.score(x_test,y_test))
print(classification_report(y_test,sgdc_y_predict,target_names=['Benign','Malignant']))

Acc of SGD Classifier 0.9415204678362573
              precision    recall  f1-score   support

      Benign       0.98      0.93      0.95       113
   Malignant       0.88      0.97      0.92        58

    accuracy                           0.94       171
   macro avg       0.93      0.95      0.94       171
weighted avg       0.95      0.94      0.94       171

